In [1]:
import lmdb
import os
import pickle
import pandas as pd

In [2]:
def merge_lmdbs(input_dir, output_name):
    
    smi_lst = []
    idx_lst = []
    total_mappig_sheet = []
    
    env_new = lmdb.open(
        output_name,
        subdir=False,
        readonly=False,
        lock=False,
        readahead=False,
        meminit=False,
        max_readers=1,
        map_size=int(500e10),
    )
    
    txn_write = env_new.begin(write=True)

    def get_lmdb_files(directories):
        lmdb_files = []
        for directory in directories:
            for root, dirs, files in os.walk(directory):
                print(f"processing: {root}")
                if any(file.endswith('.lmdb') for file in files):
                    lmdb_files.append(root)
        return lmdb_files

    def read_and_write_data(lmdb_dirs, pop_mol=False):
        count = 0
        for lmdb_dir in lmdb_dirs:
            print(f"processing: {lmdb_dir}; starting")
            temp_lmdb_dir = lmdb_dir
            temp_lmdb_file = temp_lmdb_dir + "/test.lmdb"
            current_file_count = 0
            env_read = lmdb.open(temp_lmdb_file, subdir=False, readonly=True, lock=False, readahead=False, meminit=False)
            with env_read.begin(write=False) as txn_read:
                for key, value in txn_read.cursor():
                    # drop key mol
                    value = pickle.loads(value)
                    smi_key = value["smi"]
                    if pop_mol:
                        value.pop('mol')
                    # verify the coord list
                    smi_lst.append(smi_key)
                    idx_lst.append(count)
                    total_mappig_sheet.append(str((smi_key, count, temp_lmdb_dir)))
                    value = pickle.dumps(value)
                    txn_write.put(str(count).encode("ascii"), value)
                    count += 1
                    current_file_count += 1
            print(f"finished: {lmdb_dir}; items {current_file_count}")
        return count

    lmdb_files = get_lmdb_files(input_dir)
    total_written = read_and_write_data(lmdb_files, False)

    txn_write.commit()
    env_new.close()

    print(f"Total entries written: {total_written}")
    
    # save smi and idx as txt
    output_dir = os.path.dirname(output_name)
    with open(output_dir + "/smi.txt", "w") as f:
        for item in smi_lst:
            f.write("%s\n" % item)
    with open(output_dir + "/idx.txt", "w") as f:
        for item in idx_lst:
            f.write("%s\n" % item)
    with open(output_dir + "/total_mapping_sheet.txt", "w") as f:
        for item in total_mappig_sheet:
            f.write("%s\n" % item)

dst = "/home/sdl/vector_cellxgene_data/220k-lib/all-lmdb/test.lmdb"
input_dir = ["/home/sdl/vector_cellxgene_data/220k-lib/lmdb"]

merge_lmdbs(input_dir, dst)

processing: /home/sdl/vector_cellxgene_data/220k-lib/lmdb
processing: /home/sdl/vector_cellxgene_data/220k-lib/lmdb/3
processing: /home/sdl/vector_cellxgene_data/220k-lib/lmdb/0
processing: /home/sdl/vector_cellxgene_data/220k-lib/lmdb/7
processing: /home/sdl/vector_cellxgene_data/220k-lib/lmdb/4
processing: /home/sdl/vector_cellxgene_data/220k-lib/lmdb/9
processing: /home/sdl/vector_cellxgene_data/220k-lib/lmdb/2
processing: /home/sdl/vector_cellxgene_data/220k-lib/lmdb/1
processing: /home/sdl/vector_cellxgene_data/220k-lib/lmdb/5
processing: /home/sdl/vector_cellxgene_data/220k-lib/lmdb/8
processing: /home/sdl/vector_cellxgene_data/220k-lib/lmdb/6
processing: /home/sdl/vector_cellxgene_data/220k-lib/lmdb/3; starting


finished: /home/sdl/vector_cellxgene_data/220k-lib/lmdb/3; items 22118
processing: /home/sdl/vector_cellxgene_data/220k-lib/lmdb/0; starting
finished: /home/sdl/vector_cellxgene_data/220k-lib/lmdb/0; items 22118
processing: /home/sdl/vector_cellxgene_data/220k-lib/lmdb/7; starting
finished: /home/sdl/vector_cellxgene_data/220k-lib/lmdb/7; items 22118
processing: /home/sdl/vector_cellxgene_data/220k-lib/lmdb/4; starting
finished: /home/sdl/vector_cellxgene_data/220k-lib/lmdb/4; items 22118
processing: /home/sdl/vector_cellxgene_data/220k-lib/lmdb/9; starting
finished: /home/sdl/vector_cellxgene_data/220k-lib/lmdb/9; items 22122
processing: /home/sdl/vector_cellxgene_data/220k-lib/lmdb/2; starting
finished: /home/sdl/vector_cellxgene_data/220k-lib/lmdb/2; items 22118
processing: /home/sdl/vector_cellxgene_data/220k-lib/lmdb/1; starting
finished: /home/sdl/vector_cellxgene_data/220k-lib/lmdb/1; items 22118
processing: /home/sdl/vector_cellxgene_data/220k-lib/lmdb/5; starting
finished: /ho

In [56]:
# examine the merged lmdb
def replace_repeating_backslashes(smiles):
    # find all repeating backslashes using loop
    for i in range(2,10):
        smiles = smiles.replace("\\"*i, "\\")
    return smiles

mapping_sheet_path = "/home/sdl/vector_cellxgene_data/220k-lib/all-lmdb/total_mapping_sheet.txt"
smi_list = []
with open(mapping_sheet_path, "r") as f:
    lines = f.readlines()
    smi_list = [replace_repeating_backslashes(line.split(",")[0][1:].strip("'"))
                for line in lines]
    

# check if deduplication is needed
len(smi_list) == len(set(smi_list))

True

In [57]:
len(smi_list)

221180

In [58]:
mapping_df = pd.read_csv( "/home/sdl/SDL-LNP/model/data_process/220k_library.csv")
mapping_df

,id,combined_mol_SMILES,A_smiles,B_smiles,C_smiles,D_smiles
0,0,CCCCCC(C(=O)NC)N(CCN(C)C)C(=O)CC12C[C@H]3C[C@@...,NCCN(C)C,C[N+]#[C-],CCCCCC=O,OC(CC(C1)(C2)C[C@@H]3C[C@H]2C[C@H]1C3)=O
1,1,CCCCCC(C(=O)NC)N(CCN(C)C)C(=O)C[C@@]12C[C@@H]3...,NCCN(C)C,C[N+]#[C-],CCCCCC=O,OC(C[C@@](C1)(C2)C[C@]3(O)C[C@H]2C[C@H]1C3)=O
2,2,CCCCCC(C(=O)NC)N(CCN(C)C)C(=O)CCN1CCCCC1,NCCN(C)C,C[N+]#[C-],CCCCCC=O,OC(CCN1CCCCC1)=O
3,3,CCCCCC(C(=O)NC)N(CCN(C)C)C(=O)CN(C)C,NCCN(C)C,C[N+]#[C-],CCCCCC=O,CN(CC(O)=O)C
4,4,CCCCCC(C(=O)NC)N(CCN(C)C)C(=O)CCN(C)C,NCCN(C)C,C[N+]#[C-],CCCCCC=O,CN(CCC(O)=O)C
...,...,...,...,...,...,...
221179,221179,CCCCC/C=C\C/C=C\CCCCCCCC(C(=O)NC12C[C@H]3C[C@@...,NCCCN1CCCCCC1,[C-]#[N+]C1(C[C@@H]2C3)C[C@@H]3C[C@@H](C2)C1,CCCCC/C=C\C/C=C\CCCCCCCC=O,CCCCCCCCC(OC(CCCCC(O)=O)=O)CCCCCCCC
221180,221180,CCCCC/C=C\C/C=C\CCCCCCCC(C(=O)NC12C[C@H]3C[C@@...,NCCCN1CCCCCC1,[C-]#[N+]C1(C[C@@H]2C3)C[C@@H]3C[C@@H](C2)C1,CCCCC/C=C\C/C=C\CCCCCCCC=O,CCC(CCC(OC(CCCCC(O)=O)=O)CC(C)C)CCCC
221181,221181,CCCCC/C=C\C/C=C\CCCCCCCC(C(=O)NC12C[C@H]3C[C@@...,NCCCN1CCCCCC1,[C-]#[N+]C1(C[C@@H]2C3)C[C@@H]3C[C@@H](C2)C1,CCCCC/C=C\C/C=C\CCCCCCCC=O,OC(CCCCC(OCCCCCCCCCC)=O)=O
221182,221182,CCCCC/C=C\C/C=C\CCCCCCCC(C(=O)NC12C[C@H]3C[C@@...,NCCCN1CCCCCC1,[C-]#[N+]C1(C[C@@H]2C3)C[C@@H]3C[C@@H](C2)C1,CCCCC/C=C\C/C=C\CCCCCCCC=O,OC(CCCCC(OCCCCCCCCCCC)=O)=O


In [60]:
# check non-overlapping smi
mapping_df["combined_mol_SMILES"][~mapping_df["combined_mol_SMILES"].isin(smi_list)]

221180    CCCCC/C=C\C/C=C\CCCCCCCC(C(=O)NC12C[C@H]3C[C@@...
221181    CCCCC/C=C\C/C=C\CCCCCCCC(C(=O)NC12C[C@H]3C[C@@...
221182    CCCCC/C=C\C/C=C\CCCCCCCC(C(=O)NC12C[C@H]3C[C@@...
221183    C=CCCCCCCCCCOC(=O)CCCCC(=O)N(CCCN1CCCCCC1)C(CC...
Name: combined_mol_SMILES, dtype: object

In [62]:
# reverse check
set(smi_list) - set(mapping_df["combined_mol_SMILES"])

set()

In [ ]:
# read lmdb and assert the length of the lmdb